Seq2Seq model is machine learning approach used ofr loanguage processing. It is also known as 'Encoder-Decoder Architecture'. This model architecture is useful to solve usecases where input and output are both sequence of elements. For e.g. Translation of a language to another language( say French to English) here input is english sentence which is a sequence of words and also output is french sentence a sequence of symbols.There are many such usecases like image cationing, speech processing , conversational models and text summarization etc.

A Seq2Seq model converts an input sequence to output sequence. The input and output sequences may differ in length. here is the architecture of Seq2Seq model. 
<img src='Seq2Seq-Architecture.png' style='align: right'>

The model has two core component - 'Encoder' and 'Decoder'.  
**Encoder -** The encoder component takes a sequence as an input and returns a fixed-dimensional vector containing the states of the input sequence.

**Decoder -** The decoder component turns the vector into an output sequence. Decoder component is trained on both the output sequence aswell as the fixed representation from encoder.

Encoder component is built using an Embedding layer and a RNN (specialy LSTM/GRU to handle vanishing gradient problem adn long term dependency problem).  
The Embdedding layer accepts a 2D array of input sequences , the shape of 2D array is (batch_size, sequence_length) where batch_size if number of input samples and sequence_length is maximum length of any sequence. The Embedding layer then returns a 3D array of shape(batch_size, input_length, output_dim) where input_length is maximum length of any sequence and output_dim is the fixed dimension of output vector.  

This 3D array is then passed to RNN (LSTM/GRU). The RNN process an item in the sequence to its hidden layer and predicts the next item in the sequence.This predicted output is then passed as an input to next hidden layer along with next item in the input sequence and predicts another output ans so on. finally the output of last item in the sequence is converted into a vector of fixed size. 

Decoder component is implemented using a RNN model which takes the vector returned from encoder as an input and predicts the next item in the output sequence. While training this RNN , it is trained on items of output sequence given last state from encoder step which provides context to the output sequence.

Lets learn the encoder decoder model using an example. In this example we will translate Bengali Sentences to English sentences. Data can be downloaded from http://www.manythings.org/anki/ 

So input sequence is 'Bengali Sentence' and Output sequence is 'English Sentence'.

Lets first load necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

Pair of Bengali and English senetences downloaded from the given site is in text file seperated by a tab. So we will first load the sentence pairs from the text file.

In [166]:
data = pd.read_csv('Bong-English-Sent-Pairs.txt', sep='\t', header=None)
data.head()

,0,1,2
0,Go.,যাও।,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,যান।,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,যা।,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Run!,পালাও!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
4,Run!,পালান!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


Collumn '0' corresponds to English Sentences and collumn '1' corresponds to Bengali Sentence. We do not need collumn 2 hence we will drop it.

In [167]:
data.drop(2, axis=1, inplace=True)

In [168]:
data.columns = ['English_Lang','Bengali_Lang']
data.head()

,English_Lang,Bengali_Lang
0,Go.,যাও।
1,Go.,যান।
2,Go.,যা।
3,Run!,পালাও!
4,Run!,পালান!


In [169]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4349 entries, 0 to 4348
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   English_Lang  4349 non-null   object
 1   Bengali_Lang  4349 non-null   object
dtypes: object(2)
memory usage: 68.1+ KB


In [170]:
data.isnull().sum()

English_Lang    0
Bengali_Lang    0
dtype: int64

Dataset has 4349 sentence pairs and there are no missing records. 

In [173]:
data_ = data.copy(deep=True)
data_.shape

(4349, 2)

Lets find out max length of both bengali sentence and english sentence becuase we need these to determine the sequence length.

In [174]:
max_len_bong = data_.Bengali_Lang.apply(lambda x : len([w for w in x.split(' ')])).max()
print('Max Length of Bengali Phrase : ', max_len_bong)

Max Length of Bengali Phrase :  18


In [175]:
max_len_eng = data_.English_Lang.apply(lambda x : len([w for w in x.split(' ')])).max()
print('Max Length of English Phrase : ', max_len_eng)

Max Length of English Phrase :  19


In [176]:
X = data_.Bengali_Lang
Y = data_.English_Lang

In [177]:
bong_tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=True)
bong_tokenizer.fit_on_texts(X)

eng_tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=True)
eng_tokenizer.fit_on_texts(Y)

In [179]:
bong_vocb_size = len(bong_tokenizer.word_index) + 1
print('Bengali Vocabulary Length ', bong_vocb_size)

Bengali Vocabulary Length  3321


In [180]:
eng_vocab_size =len(eng_tokenizer.word_index) + 1
print('English Vocabulary Length ', eng_vocab_size)

English Vocabulary Length  1876


I am not going to perform any kind of NLP preprocessing techniques such as stop word removals, stemming, lemmatization because this usecase is a language translation usecase. Each word in a sentence matters.
Next lets split our data set into training and testing set. Remember X is Bengali Sentence and Y is English Sentence

In [181]:
from sklearn.model_selection import train_test_split
x_train,x_test , y_train, y_test = train_test_split(X, Y , test_size=0.1, random_state=42)
print(x_train.shape , x_test.shape , y_train.shape, y_test.shape)

(3914,) (435,) (3914,) (435,)


I will now prepare the text data into sequence of integers where each integer corresponds to index of a word in that sequence. Because we can not directly pass text data to RNN. first we need to convert it into sequence . 
Keras Tokenization helps to converts sentences into sequence of integers. And once sequence is ready i will us 'pad_sequences' to padd 0 for shorter or bigger legth sequences. We need to convert both input and output sentences imto corresponding sequences.

In [183]:
# Russian Sentences to Sequence
x_train_seq = bong_tokenizer.texts_to_sequences(x_train)
x_train_seq = tf.keras.preprocessing.sequence.pad_sequences(x_train_seq, maxlen= max_len_bong, padding='post', truncating='post')



# English Sentences to Sequence
y_train_seq = eng_tokenizer.texts_to_sequences(y_train)
y_train_seq = tf.keras.preprocessing.sequence.pad_sequences(y_train_seq, maxlen= max_len_eng, padding='post', truncating='post')


I have used lower=True in the Tokenizer class , to allow lowering the words in a sentence. in pad_sequences i have used maxlen= max_len_bong which is 18 incase of bengali sentences and 19 incase of english sentences. maxlen tells what maximum length to consider for a sequence. if any sequence is shorter maxlen then 0 would be padded and if any sequence is bigger than maxlen then items in that sequence would be trimed off.   

Vocabulary size is nothing but total unique words in whole text corpus. We can see consider whole english sentences gives 1876 unique words. and bengali sentences have overall 3321 unique words. This vocabulary size is required in Embedding layer because Embedding layer needs to understand for how many words/items embeddings have to be generated. 

In [184]:
print(x_train_seq.shape, y_train_seq.shape)

(3914, 18) (3914, 19)


Now we have both input and output sequences are ready. Input sequences are sequences of integers corresponds to each word in corresponding bengali sentence. and output sequences are sequences of integers coreesponds to words in english sentence. We are ready to build Encoder-Decoder model or Seq2Seq model.

Encoder:
We will first use Embedding layer to generate word embeddings for each word in whole input sentences. 
Next these word embeddings will be sent to LSTM model to save the state and return a fixed size vector of states from item in a sequence.

Decoder:
We will send these fixed sized vector of states from encoder into another LSTM model which will get trained on output sequences conditioned context passed from encoder.

In [185]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim= bong_vocb_size, output_dim=512, input_length=max_len_bong))
model.add(tf.keras.layers.LSTM(units=512))
model.add(tf.keras.layers.RepeatVector(max_len_eng))
model.add(tf.keras.layers.LSTM(units=512, return_sequences=True))
model.add(tf.keras.layers.Dense(eng_vocab_size, activation='softmax'))

In the above code, the first step involves creating a keras sequential model.   

The first layer in the Encoder component is a Embedding Layer , This will convert our words (referenced by integers in the data) into meaningful embedding vectors. This Embedding() layer takes the size of the vocabulary as its first argument, then the size of the resultant embedding vector that you want as the next argument. Finally, because this layer is the first layer in the network, we must specify the “length” of the input i.e. the number of steps/words in each sample. In this example shape of embedding   

The next layer is the first of our two LSTM layers. To specify an LSTM layer, first we have to provide the number of nodes in the hidden layers within the LSTM cell,e.g. the number of cells in the forget gate layer, the tanh squashing input layer and so on. The next argument that is specified in the code above is the return_sequences=False argument. This argument ensures that LSTM cell returns the output of LSTM cell from last time step only. return_sequences=True ensures LSTM cell returns outputs from unrolled LSTM cell through all time stamps. As we have seen architecture diagram , only the output of last time step is given to decoder component , hence we have set return_sequences=False. <img src='Se2Seq-Return_Seq.png'>

A LSTM has as many cells as timesteps. The second LSTM would be having 19 cells as we have maximum 19 timesteps for english phrases. Lets understand why do we need RepeatVector . In the Encoder-Decoder model we pass the output of the last timestep in the encoder LSTM layer, to the LSTM layer in Decoder component. Now The output from encoder component needs to be passed to each and evry timestep in Decoder LSOM layer. But we get oly a single vector in encoder's LSTM layer. How do we pass this single output vector to every timestep in next LSTM layer. Answer is RepeatVector. We repeat the same output vector as many times as the next LSTM layer has time stpes. In our usecase next LSTM layer (Decoder Comp) will have 19 timesteps because this LSTM will be trained on output sequence and output sequence is engalish sentence which has max 19 length.That is why we have set RepeatVector(max_len_eng=19) so that the output of LSTM layer in Encoder component is passed to every 19 timesteps in next LSTM layer. Below figure will clear it better. <img src='Repeat-Vector.png'>  

Next layer is LSTM layer of Decoder component which will get trained on output sequence i.e eglish language sequence given the output of last timestep in the LSTM layer of Encoder. We are predicting the English words corresponds to input bengali words. so we need output from all timesteps in this LSTM layer. hence we have set return_sequences=True.

Next layer is Dense layer or fully connected layer of length english vocabulary size. This layer is used with an activation of Softmax which will quash the output between 0 and 1.

Next we will compile the model with optimizer='rmsprop' and loss='sparse_categorical_crossentropy'. Our usecase is to translate bengali sentences into english senetences. It is basically a multiclass classification problem where the model predicts the english word in the target english vocabulary given the russian sequence as input. We will use loss='sparse_categorical_crossentropy.

In [186]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Lets train the model with input sequences (x_train_seq) and output sequences (y_train_seq) . We will train our model with an epoch=200 and batch_size=512 and a validation_split=0.2.  

Epoch is a hyperparemeter which specifies how many times entire dataset is passed forward and backward through the neural network only once. here we want entire dataset to be used 200 times for training the model.  

batch_size = After how many training samples the gradient to be updated. Here batch_size=512 menas after every 512 samples in a epoch the model will update the weights through gradient descent.  

validation_split=0.2 means 20% of training samples to be used for validation purpose and remaining 80% to be used for training the model.

In [187]:
y_train_seq = y_train_seq.reshape(y_train_seq.shape[0], y_train_seq.shape[1], 1)
y_train_seq.shape

(3914, 19, 1)

In [120]:
model.fit(x_train_seq, y_train_seq, epochs=200, batch_size=512, validation_split = 0.2, verbose=1)

Train on 3131 samples, validate on 783 samples
Epoch 1/200
3131/3131 [==============================] - 74s 24ms/sample - loss: 3.6601 - accuracy: 0.6375 - val_loss: 1.8357 - val_accuracy: 0.7561
Epoch 2/200
3131/3131 [==============================] - 64s 20ms/sample - loss: 1.7726 - accuracy: 0.7613 - val_loss: 1.7244 - val_accuracy: 0.7638
Epoch 3/200
3131/3131 [==============================] - 62s 20ms/sample - loss: 1.6840 - accuracy: 0.7679 - val_loss: 1.6927 - val_accuracy: 0.7635
Epoch 4/200
3131/3131 [==============================] - 64s 20ms/sample - loss: 1.6143 - accuracy: 0.7700 - val_loss: 1.8136 - val_accuracy: 0.7627
Epoch 5/200
3131/3131 [==============================] - 64s 20ms/sample - loss: 1.5871 - accuracy: 0.7715 - val_loss: 1.6429 - val_accuracy: 0.7676
Epoch 6/200
3131/3131 [==============================] - 63s 20ms/sample - loss: 1.5700 - accuracy: 0.7719 - val_loss: 1.6518 - val_accuracy: 0.7676
Epoch 7/200
3131/3131 [==============================] - 63

3131/3131 [==============================] - 70s 22ms/sample - loss: 1.1043 - accuracy: 0.7908 - val_loss: 1.4700 - val_accuracy: 0.7776
Epoch 56/200
3131/3131 [==============================] - 102s 33ms/sample - loss: 1.1086 - accuracy: 0.7905 - val_loss: 1.4906 - val_accuracy: 0.7807
Epoch 57/200
3131/3131 [==============================] - 79s 25ms/sample - loss: 1.0910 - accuracy: 0.7913 - val_loss: 1.4821 - val_accuracy: 0.7810
Epoch 58/200
3131/3131 [==============================] - 102s 33ms/sample - loss: 1.0687 - accuracy: 0.7932 - val_loss: 1.4973 - val_accuracy: 0.7759
Epoch 59/200
3131/3131 [==============================] - 87s 28ms/sample - loss: 1.0884 - accuracy: 0.7919 - val_loss: 1.4780 - val_accuracy: 0.7797
Epoch 60/200
3131/3131 [==============================] - 87s 28ms/sample - loss: 1.0634 - accuracy: 0.7927 - val_loss: 1.5438 - val_accuracy: 0.7809
Epoch 61/200
3131/3131 [==============================] - 71s 23ms/sample - loss: 1.0616 - accuracy: 0.7940 - v

3131/3131 [==============================] - 64s 20ms/sample - loss: 0.7141 - accuracy: 0.8332 - val_loss: 1.4574 - val_accuracy: 0.7959
Epoch 110/200
3131/3131 [==============================] - 63s 20ms/sample - loss: 0.7171 - accuracy: 0.8330 - val_loss: 1.4498 - val_accuracy: 0.7935
Epoch 111/200
3131/3131 [==============================] - 65s 21ms/sample - loss: 0.7078 - accuracy: 0.8345 - val_loss: 1.4297 - val_accuracy: 0.7942
Epoch 112/200
3131/3131 [==============================] - 65s 21ms/sample - loss: 0.7096 - accuracy: 0.8342 - val_loss: 1.4717 - val_accuracy: 0.7928
Epoch 113/200
3131/3131 [==============================] - 64s 20ms/sample - loss: 0.6844 - accuracy: 0.8383 - val_loss: 1.4191 - val_accuracy: 0.7960
Epoch 114/200
3131/3131 [==============================] - 64s 20ms/sample - loss: 0.6779 - accuracy: 0.8402 - val_loss: 1.4260 - val_accuracy: 0.7956
Epoch 115/200
3131/3131 [==============================] - 64s 20ms/sample - loss: 0.6802 - accuracy: 0.8376

3131/3131 [==============================] - 64s 20ms/sample - loss: 0.3429 - accuracy: 0.9108 - val_loss: 1.4262 - val_accuracy: 0.8110
Epoch 164/200
3131/3131 [==============================] - 64s 20ms/sample - loss: 0.3184 - accuracy: 0.9192 - val_loss: 1.4264 - val_accuracy: 0.8173
Epoch 165/200
3131/3131 [==============================] - 64s 21ms/sample - loss: 0.3322 - accuracy: 0.9142 - val_loss: 1.4032 - val_accuracy: 0.8133
Epoch 166/200
3131/3131 [==============================] - 66s 21ms/sample - loss: 0.3245 - accuracy: 0.9165 - val_loss: 1.4032 - val_accuracy: 0.8122
Epoch 167/200
3131/3131 [==============================] - 67s 21ms/sample - loss: 0.3123 - accuracy: 0.9202 - val_loss: 1.4111 - val_accuracy: 0.8197
Epoch 168/200
3131/3131 [==============================] - 63s 20ms/sample - loss: 0.2941 - accuracy: 0.9249 - val_loss: 1.4194 - val_accuracy: 0.8166
Epoch 169/200
3131/3131 [==============================] - 63s 20ms/sample - loss: 0.3131 - accuracy: 0.9194

Our Seq2Seq model has achieved an accuracy of 95%. We can increase this by training model on more traning samples.  

Now we will use this trained encoder-decoder model to translate a given bengali sentence in the tseting data set into english sentence.  First we will convert test bengali sentences into sequence of integers.

In [122]:
x_test_seq = bong_tokenizer.texts_to_sequences(x_test)
x_test_seq = tf.keras.preprocessing.sequence.pad_sequences(x_test_seq, maxlen= max_len_rus,padding='post',truncating='post')


In [125]:
x_test_seq

array([[  95,  100,   30, ...,    0,    0,    0],
       [  10,   31,   41, ...,    0,    0,    0],
       [  10,  970,   81, ...,    0,    0,    0],
       ...,
       [   1,  367, 2319, ...,    0,    0,    0],
       [  42,   37,  171, ...,    0,    0,    0],
       [   4, 1723,    0, ...,    0,    0,    0]])

We will predict output sequence passing x_test_seq into predict_classes method of model. model also has predict method but it would give the probability value of each class. where as predict_classes gives class with highest prob value.

In [126]:
preds = model.predict_classes(x_test_seq)

In [128]:
preds

array([[  12,    5,   50, ...,    0,    0,    0],
       [  10,    3,   54, ...,    0,    0,    0],
       [  90,   16,  422, ...,    0,    0,    0],
       ...,
       [   1,   19,    7, ...,    0,    0,    0],
       [   2,  181,  155, ...,    0,    0,    0],
       [  20,  847, 1176, ...,    0,    0,    0]], dtype=int64)

Now we have predicted output sequence. Lets get corresponding sentence for each predicted sequence

In [154]:
words = list(eng_tokenizer.word_index.keys())
predicted_value = [' '.join([words[y-1] for y in y_pred if y>0]) for y_pred in preds]


In [164]:
df = pd.DataFrame({
    'Input': x_test,
    'Actual': y_test,
    'Predicted': predicted_value
})

In [165]:
df.iloc[100:120,:]

,Input,Actual,Predicted
1210,আমি কখনই বলবো না।,I'll never tell.,i have learning french
1200,আমি আপনার সহায়তা করবো।,I'll assist you.,i'm your friend
2247,নিজেকে বিভ্রান্ত কোরো না।,Don't delude yourself.,don't delude yourself
1257,সে কি শিক্ষক?,Is he a teacher?,is he a teacher
1075,আপনারা কি ডাক্তার?,Are you doctors?,are you a doctor
1116,সবাই এটা দেখেছিলো।,Everyone saw it.,everybody saw it
1052,আমরা ডাক্তার।,We are doctors.,we're are
3004,তুই কি টমের উপর নজর রাখতে পারবি?,Can you keep an eye on Tom?,can you keep an eye on tom
2924,আমি এই ব্যাপারে খুব একটা ভালো নই।,I'm not very good at this.,i finally found my my well
621,টম আহত।,Tom is hurt.,tom's injured


We can see in the above dataframe model has done quiet well to predict english sequence for input bengali sequence.